In [12]:
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
import shapely.geometry as sg
import shapely.ops as so

DIR = Path('..')
sys.path.append(str(DIR))

import gtfs_kit as gk

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
path = DATA_DIR/'cairns_gtfs.zip'
feed = gk.read_gtfs(path, dist_units='km')


In [14]:
trip_ids = feed.trips.trip_id.loc[:1]
print(trip_ids)
g = feed.geometrize_stops()
display(g)
type(g)

0    CNS2014-CNS_MUL-Weekday-00-4165878
1    CNS2014-CNS_MUL-Weekday-00-4165879
Name: trip_id, dtype: object


,stop_id,stop_code,stop_name,stop_desc,zone_id,stop_url,location_type,parent_station,geometry
0,750000,NaN,Cedar Rd (Palm Cove) - Hail and Ride Location,NaN,NaN,NaN,0,NaN,POINT (145.668217 -16.74359)
1,750001,NaN,Williams Esplanade N201,NaN,NaN,NaN,0,NaN,POINT (145.67111 -16.744015)
2,750002,NaN,Talpa Close - Hail and Ride Location,NaN,NaN,NaN,0,NaN,POINT (145.667893 -16.749258)
3,750003,NaN,Veivers Road N203,NaN,NaN,NaN,0,NaN,POINT (145.663675 -16.748213)
4,750004,NaN,Captain Cook Hwy N3 (Cairns Tropical Zoo),NaN,NaN,NaN,0,NaN,POINT (145.663283 -16.757931)
...,...,...,...,...,...,...,...,...,...
411,750452,NaN,The Pier Cairns - Terminus Stop B,NaN,NaN,NaN,0,NaN,POINT (145.778614 -16.920632)
412,750453,NaN,The Pier Cairns - Terminus Stop C,NaN,NaN,NaN,0,NaN,POINT (145.778913 -16.920741)
413,750454,NaN,The Pier Cairns - Terminus Stop D,NaN,NaN,NaN,0,NaN,POINT (145.77907 -16.920814)
414,750455,NaN,Sunbus Depot - Dunne Street,NaN,NaN,NaN,0,NaN,POINT (145.703608 -16.824684)


geopandas.geodataframe.GeoDataFrame

In [ ]:
def geometrize_routes(feed, route_ids=None):
    """
    """
    if feed.shapes is None:
        return
    
    # Subset routes
    routes = feed.routes.copy()
    if route_ids is not None:
        routes = routes.loc[lambda x: x.route_id.isin(route_ids)].copy()
        
    # Subset shapes
    trips = feed.trips.merge(routes.filter(["route_id"]), how="right")
    shapes = feed.shapes.merge(trips, how="right")

    # Combine shape LineStrings within route and direction
    def combine_shapes(group):
        d = {}
        d["geometry"] = so.linemerge(group.geometry.tolist())
        return pd.Series(d)
    
    g = (
        gk.geometrize_shapes(shapes)
        .merge(trips)
        .filter(["route_id", "direction_id", "geometry"])
        .groupby(["route_id", "direction_id"])
        .apply(combine_shapes)
        .reset_index()
        .merge(routes)
    )

    return g
    
        

In [ ]:
route_ids = feed.routes.route_id.values[:2]
%time routes_g = geometrize_routes(feed)

In [ ]:
routes_g.geometry.iat[0]

In [ ]:
# Slicing 
display(fts.loc[:, ("num_trips", slice(None))])
display(fts.xs("num_trips", axis="columns"))

In [ ]:
f = fts = feed.compute_feed_time_series(trip_stats, dates[:1], freq="12H", split_route_types=False)
display(f)
g = gk.unstack_time_series(f)

# columns = [c for c in g.columns if c not in ["datetime", "value"]]
# h = g.pivot_table(index="datetime", columns=columns).value.sort_index(
#     axis="columns"
# )

# hours = (h.index[1] - h.index[0]).components.hours
# if hours != 0:
#     freqy = f"{hours}H"
# else:
#     freqy = "D"

# print(freqy)

h = gk.restack_time_series(g)
h

In [ ]:
f.index

In [ ]:
delta.components.hours

In [ ]:
feed1 = feed.copy()
cal = feed1.calendar.copy()
cal["monday"] = 0
feed1.calendar = cal
feed1.compute_feed_time_series(trip_stats, dates, freq="12H")

In [ ]:
s1 = pd.Series({"hello": 12, "goodbye": 13})
s2 = s1.copy()
pd.DataFrame([s1, s2])

In [ ]:
feed.compute_stop_stats(['20140601', '20140603'])

In [ ]:
trip_stats = feed.compute_trip_stats()
feed = feed.append_dist_to_stop_times(trip_stats)

# Load screen line
with (DATA_DIR/'cairns_screen_line.geojson').open() as src:
    line = json.load(src)
    line = sg.shape(line['features'][0]['geometry'])



In [ ]:
f = feed.compute_screen_line_counts(line, dates[:7])
f[f['crossing_time'] < '06:00:00']

In [ ]:
feed.compute_feed_time_series(ts, dates[-2:], freq='12H')

In [ ]:
sd = False
#%time p1 = proto1(feed, ts, dates[:14], split_directions=sd)
%time p2 = proto2(feed, [dates[0], '20010101'], split_directions=sd, freq='12H')
p2

In [ ]:
feed.compute_feed_stats(ts, dates[0])

In [ ]:
feed.describe()

In [ ]:
feed.summarize()